In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 21136082
paper_name = 'villa_garcia_henry_2011' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/NIHMS265254-supplement-2.xlsx', sheet_name='Table 1')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 448 x 5


In [7]:
original_data.head()

,ORF,Gene,I-C-\n30º,I-C+\n30º,I-C+\n37º
0,YAL021C,CCR4,+,+,W
1,YAL023C,PMT2,W,W,S
2,YAL024C,LTE1,W,W,W
3,YAL026C,DRS2,+,+,S
4,YAL040C,CLN3,+,+,W


In [8]:
original_data['orf'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, Gene, I-C-
30º, I-C+
30º, I-C+
37º, orf]
Index: []


In [12]:
original_data.set_index('orf', inplace=True)

In [13]:
original_data = original_data[['I-C-\n30º','I-C+\n30º','I-C+\n37º']].copy()

In [14]:
for c in original_data.columns:
    original_data[c] = original_data[c].str.upper()

In [19]:
switch_data = {'+': 0, 'VW': -1, 'W': -2, 'S': -3, 'NS': np.nan, 'NG': np.nan}
for c in original_data.columns:
    original_data[c] = original_data[c].apply(lambda x: switch_data[x])

In [21]:
original_data = original_data.groupby(original_data.index).mean()

In [22]:
original_data.shape

(448, 3)

# Prepare the final dataset

In [23]:
data = original_data.copy()

In [24]:
dataset_ids = [16191,16192,16193]
datasets = datasets.reindex(index=dataset_ids)

In [25]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [26]:
data.head()

dataset_id,16191,16192,16193
data_type,value,value,value
orf,,,
YAL021C,0.0,0.0,-2.0
YAL023C,-2.0,-2.0,-3.0
YAL024C,-2.0,-2.0,-2.0
YAL026C,0.0,0.0,-3.0
YAL040C,0.0,0.0,-2.0


## Subset to the genes currently in SGD

In [27]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 0


In [28]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,16191,16192,16193
,data_type,value,value,value
gene_id,orf,,,
19,YAL021C,0.0,0.0,-2.0
21,YAL023C,-2.0,-2.0,-3.0
22,YAL024C,-2.0,-2.0,-2.0
24,YAL026C,0.0,0.0,-3.0
38,YAL040C,0.0,0.0,-2.0


# Normalize

In [29]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [30]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [31]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,16191,16192,16193,16191,16192,16193
,data_type,value,value,value,valuez,valuez,valuez
gene_id,orf,,,,,,
19,YAL021C,0.0,0.0,-2.0,0.000000,0.000000,-2.563460
21,YAL023C,-2.0,-2.0,-3.0,-5.363968,-4.502446,-3.845189
22,YAL024C,-2.0,-2.0,-2.0,-5.363968,-4.502446,-2.563460
24,YAL026C,0.0,0.0,-3.0,0.000000,0.000000,-3.845189
38,YAL040C,0.0,0.0,-2.0,0.000000,0.000000,-2.563460


# Print out

In [32]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [33]:
from IO.save_data_to_db3 import *

In [34]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/3 [00:00<?, ?it/s]

Deleting all datasets for PMID 21136082...
Inserting the new data...


100%|██████████| 3/3 [00:02<00:00,  1.36it/s]

Updating the data_modified_on field...
